# Lab_4
##### 1. Провести моделирование регрессионного процесса с гетероскедастичным возмущением.
#####  2. Полученные данные проверить по тестам на наличие гетероскедастичности.
#####  3. Провести оценивание параметров регрессионной модели по доступному обобщенному МНК и по обыкновенному МНК.
#####  4. Сравнить эффективность оценок в этих двух случаях по квадрату их расстояния до известных истинных значений параметров. 

## Инициализация программы

In [1]:
import numpy as np
import random as rnd
import math

f = lambda x1, x2 : np.array([1, 1/x1, x1, x2, x2**2]) 
teta = np.array([0.001, 1, 0.001, 0.0001, 1])
u = lambda x1, x2: 0.001 + 1/x1 + x1* 0.001 + x2*0.0001 + x2**2

# Генерирует точки (120)
def generateData():
    import random as rnd

    x1_points = []
    x2_points = []

    for s in range(0, 120):
        x2_points.append((rnd.random() - 0.5) * 2)

    for s in range(0, 60):
        x1_points.append(-rnd.random() * 2 - 1)
        x1_points.append(rnd.random() * 2 + 1)
        
    x1_points = np.array(x1_points)
    x2_points = np.array(x2_points)
    return x1_points, x2_points

# Строит модель из 2 лабы
def lab_2(f, teta, u, x1_array, x2_array, p):   
    vector_u = list()

    # Рассчиатли U
    for i in range(len(x1_array)):
        vector_u.append(u(x1_array[i], x2_array[i]))

    vect_arr_u = np.array(vector_u)
    average_u = vect_arr_u.mean()

    w2 = np.dot(vect_arr_u - average_u, vect_arr_u - average_u) / (len(vect_arr_u - 1))  
    sig = p * (w2)**0.5

    # Сгенерировали U с помехами (y)
    y = []
    for i in range(len(x1_array)):
        y.append(vector_u[i] + np.random.normal(0, sig))
    y = np.array(y)
    
    
    # Построение матрицы X
    X = []
    for i in range(len(teta)):
        X.append([])
        for j in range(len(x1_array)):
            X[i].append(f(x1_array[j], x2_array[j])[i])

    X = np.matrix(X)

    Q_ = X.dot(X.T)
    Q_ = np.linalg.inv(Q_)
    Q_ = Q_.dot(X)
    Q_ = Q_ * np.matrix(y).T
    
    e = []
    e = y - (X.T).dot(np.matrix(teta).T).T
    sigma2 = float(e.dot(e.T)/(len(vect_arr_u)-len(teta))) # Несмещенная оценка оценка дисперсии

    F = sigma2 / sig**2
        
    return Q_, F, X, sigma2, y, vector_u

# Генерация гетероскедатичности
def genGeteroSk(u_arr):
    y = []
    for i in range(len(u_arr)):
        y.append(u_arr[i] + np.random.normal(0, math.exp(abs(u_arr[i]))))
    y = np.array(y)
    return y
    
# Функция для генерации вектора известных переменных
def someVal(x):
    l = [];
    for i in range(len(x)):
        l.append(float(math.exp(abs(x[i]))))
    l = np.array(l, dtype = float)
    return l

## 1. Провести моделирование регрессионного процесса с гетероскедастичным возмущением.

In [2]:
# Генерация точек наблюдения и данных из 2 лабораторной работы
x1_arr, x2_arr = generateData()
Q_, F, X, sigma2, y, u_arr = lab_2(f, teta, u, x1_arr, x2_arr, 0.07)

# Генерация гетероскедатичности
y = genGeteroSk(u_arr)

## 2. Полученные данные проверить по тестам на наличие гетероскедастичности.
### 1) Тест Бреуша-Пагана

In [3]:
# МНК
X = []
for i in range(len(teta)):
    X.append([])
    for j in range(len(x1_arr)):
        X[i].append(f(x1_arr[j], x2_arr[j])[i])

X = np.matrix(X)

Q_ = X.dot(X.T)
Q_ = np.linalg.inv(Q_)
Q_ = Q_.dot(X)
Q_ = Q_ * np.matrix(y).T
Q_ = np.array(Q_)

In [4]:
def Ufunc(x1, x2, Q_oc):
    return Q_oc[0] + 1/x1 * Q_oc[1] + x1* Q_oc[2] + x2*Q_oc[3] + Q_oc[4]*x2**2

e_t = np.array(y) - Ufunc(x1_arr, x2_arr, Q_) 
sigma_tilda_2 = sum(e_t**2) / len(e_t) 

c_t = e_t**2/sigma_tilda_2 

# Строим регрессию
c_vector = np.matrix(c_t)
z = np.matrix([np.ones(len(y)), someVal(y)])
alpha = (np.linalg.inv(z*z.T)*z)*c_vector.T

# По этой регрессии оцениваем ESS
c_i = z.T * alpha 
print(alpha)
c_mean = c_i.mean()

ESS = 0
for i in range(len(c_i)):
    ESS += (c_i[i] - c_mean)**2

from scipy.stats import chi2
value = chi2.ppf(0.95, 2)
if ESS/2 < value:
    print('Гипотеза о гомоскедастичности принимается.')
else:
    print('Гипотеза о гомоскедастичности отвергается.')

[[0.81816765]
 [0.00157011]]
Гипотеза о гомоскедастичности отвергается.


### 2) Тест Голдфельда-Квандтона

In [5]:
import pandas as pd

s = someVal(y)    
values = pd.DataFrame({'x1': x1_arr, 'x2': x2_arr, 'y': y, 'u': u_arr, 'exp(|y|)': s})

# Сортировка по возрастанию 
values = values.sort_values('exp(|y|)')
values = values.reset_index(drop=True)

In [6]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
nc = len(x1_arr)/3

# Оценка по первым (n-nc)/2 наблюдений
newTable = values
X = []
for i in range(len(teta)):
    X.append([])
    for j in range(40):
        X[i].append(f(newTable['x1'][j],newTable['x2'][j])[i])
X = np.matrix(X)

Q_T = np.linalg.inv(X.dot(X.T))
Q_T = Q_T.dot(X)
Q_T = Q_T.dot(np.matrix(newTable['y'][0:40]).T)

RSS1=(np.array(newTable['y'][0:40])-X.T.dot(Q_T).T)
RSS1=float(RSS1.dot(RSS1.T))

# Оценка по последним (n-nc)/2 наблюдений
n = len(x1_arr)
X = []

for i in range(len(teta)):
    X.append([])
    for j in range(40):
        X[i].append(f(newTable['x1'][j+80],newTable['x2'][j+80])[i])
X = np.matrix(X)

Q_T = np.linalg.inv(X.dot(X.T))
Q_T = Q_T.dot(X)

newTable = newTable.reset_index(drop=True)
Q_T = Q_T.dot(np.matrix(newTable.y[80:]).T)
RSS2=(np.array(newTable['y'][80:])-X.T.dot(Q_T).T)
RSS2=float(RSS2.dot(RSS2.T))

from scipy.stats import f
k = 5
value = f.ppf(0.95, (n - nc - 2*k)/2, (n - nc - 2*k) / 2)
if RSS2/RSS1 < value:
    print('Гипотеза о гомоскедастичности принимается.')
else:
    print('Гипотеза о гомоскедастичности отвергается.')

Гипотеза о гомоскедастичности отвергается.


## 3. Провести оценивание параметров регрессионной модели по доступному обобщенному МНК и по обыкновенному МНК.

In [7]:
# ОМНК
f = lambda x1, x2 : np.array([1, 1/x1, x1, x2, x2**2]) 

X_new = []
for i in range(len(teta)):
    X_new.append([])
    for j in range(len(x1_arr)):
        X_new[i].append(f(x1_arr[j], x2_arr[j])[i])

X_new = np.matrix(X_new).T
V = np.matrix(np.diag(someVal(y)))
Q_1 = (X_new.T).dot(np.linalg.inv(V))
Q_1 = np.linalg.inv(Q_1.dot(X_new))
Q_1 = Q_1.dot((X_new.T).dot(np.linalg.inv(V)))
Q_1 = Q_1.dot(np.matrix(y).T)
Q_1 = np. array(Q_1)

In [8]:
print('Истинные значения', teta)
print('МНК', Q_.T)
print('ОМНК', Q_1.T)

Истинные значения [1.e-03 1.e+00 1.e-03 1.e-04 1.e+00]
МНК [[0.32040914 0.87305522 0.01053191 0.06160919 0.20865515]]
ОМНК [[ 0.21204033  0.13113445 -0.04162038  0.07248972 -0.51095332]]


## 4. Сравнить эффективность оценок в этих двух случаях по квадрату их расстояния до известных истинных значений параметров.

In [9]:
print('Сравнение МНК с истиной', (teta - Q_.T).dot((teta - Q_.T).T))

Сравнение МНК с истиной [[0.74823808]]


In [10]:
print('Сравнение ОМНК с истиной', (teta - Q_1.T).dot((teta - Q_1.T).T))

Сравнение ОМНК с истиной [[3.08950207]]
